In [1]:
import keras
import tensorflow as tf
from keras.applications import VGG16
from keras.utils.vis_utils import plot_model
import os, shutil
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras import models
from keras.preprocessing import image
import numpy as np
import sklearn.metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import itertools
import seaborn
print(keras.__version__)
print(tf.__version__)

2.4.3
2.4.1


In [2]:
conv_base = VGG16(weights='imagenet',include_top=False,input_shape=(150, 150, 3))

In [3]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [4]:
plot_model(conv_base, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [5]:
original_dataset_dir_rust = 'RustPhotos2'
original_dataset_dir_norust = 'norust'

#dir to store images
store_directory = 'ttv_images'
train_dir = os.path.join(store_directory, 'train')
validation_dir = os.path.join(store_directory, 'validation')
test_dir = os.path.join(store_directory, 'test')

rust_train = os.path.join(train_dir, 'rust')
norust_train = os.path.join(train_dir, 'norust')

norust_validation = os.path.join(validation_dir, 'norust')
rust_validation = os.path.join(validation_dir, 'rust')

rust_test = os.path.join(test_dir, 'rust')
norust_test = os.path.join(test_dir, 'norust')



In [9]:

##runs a single time
os.mkdir(store_directory)
os.mkdir(train_dir)
os.mkdir(validation_dir)
os.mkdir(test_dir)
os.mkdir(rust_train)
os.mkdir(norust_train)
os.mkdir(rust_validation)
os.mkdir(norust_validation)
os.mkdir(rust_test)
os.mkdir(norust_test)

In [20]:
fnames =  os.listdir(original_dataset_dir_norust)
print(len(fnames))

72


In [23]:
# #copies first 75 images to train 
filenames =  os.listdir(original_dataset_dir_rust)
# for i in range(0, 70):
#     src = os.path.join(original_dataset_dir_rust, filenames[i])
#     dst = os.path.join(rust_train, filenames[i])
#     shutil.copyfile(src, dst)

# #copies first 5 images to train 

for i in range(69, 75):
    src = os.path.join(original_dataset_dir_rust, filenames[i])
    dst = os.path.join(rust_validation, filenames[i])
    shutil.copyfile(src, dst)

#copies first 5 images to train 
for i in range(70, 79):
    src = os.path.join(original_dataset_dir_rust, filenames[i])
    dst = os.path.join(rust_test, filenames[i])
    shutil.copyfile(src, dst)

filenames =  os.listdir(original_dataset_dir_norust)
# for i in range(0, 60 ):
#     src = os.path.join(original_dataset_dir_norust, filenames[i])
#     dst = os.path.join(norust_train, filenames[i])
#     shutil.copyfile(src, dst)

for i in range(61, 67 ):
    src = os.path.join(original_dataset_dir_norust, filenames[i])
    dst = os.path.join(norust_validation, filenames[i])
    shutil.copyfile(src, dst)

for i in range(63, 71 ):
    src = os.path.join(original_dataset_dir_norust, filenames[i])
    dst = os.path.join(norust_test, filenames[i])
    shutil.copyfile(src, dst)


In [6]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))




In [7]:
model.summary()
conv_base.trainable = False 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [8]:
base_dir = 'ttv_images'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=4,
        shuffle=True,
        class_mode='binary')


validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=4,
        shuffle=True,
        class_mode='binary')

Found 130 images belonging to 2 classes.
Found 12 images belonging to 2 classes.


In [9]:
from time import time
from keras.callbacks import TensorBoard

# tensorboard = keras.callbacks.TensorBoard(log_dir='/output/{}'.format(time()))

model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=2e-5),metrics=['acc'])


history = model.fit_generator(train_generator,steps_per_epoch=10,epochs=15,validation_data=validation_generator,validation_steps=20,verbose=2)

Epoch 1/15
10/10 - 29s - loss: 0.6089 - acc: 0.7250 - val_loss: 0.5247 - val_acc: 0.8333
Epoch 2/15
10/10 - 12s - loss: 0.5054 - acc: 0.8750
Epoch 3/15
10/10 - 13s - loss: 0.4707 - acc: 0.8500
Epoch 4/15
10/10 - 12s - loss: 0.4187 - acc: 0.8500
Epoch 5/15
10/10 - 10s - loss: 0.3605 - acc: 0.9474
Epoch 6/15
10/10 - 12s - loss: 0.3741 - acc: 0.8684
Epoch 7/15
10/10 - 11s - loss: 0.2446 - acc: 0.9500
Epoch 8/15
10/10 - 11s - loss: 0.2895 - acc: 0.9211
Epoch 9/15
10/10 - 11s - loss: 0.3716 - acc: 0.8421
Epoch 10/15
10/10 - 13s - loss: 0.2101 - acc: 1.0000
Epoch 11/15
10/10 - 9s - loss: 0.3324 - acc: 0.8500
Epoch 12/15
10/10 - 13s - loss: 0.3043 - acc: 0.8750
Epoch 13/15
10/10 - 13s - loss: 0.3508 - acc: 0.8500
Epoch 14/15
10/10 - 11s - loss: 0.2724 - acc: 0.9211
Epoch 15/15
10/10 - 12s - loss: 0.2326 - acc: 0.9250


In [10]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=4,
        shuffle=False, 
        class_mode='binary')

test_loss, test_acc = model.evaluate_generator(test_generator, steps=10)
print('test acc:', test_acc)

Found 17 images belonging to 2 classes.
test acc: 0.7058823704719543


In [14]:
test_data_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=4,
        shuffle=False, # v imp : do not shuffle in case of test data, when measuring precision and recall
        class_mode='binary')

test_steps_per_epoch = np.math.ceil(test_data_generator.samples / test_data_generator.batch_size)

predictions = model.predict_generator(test_data_generator, steps=test_steps_per_epoch)

val_preds = np.zeros((predictions.shape[0],1))
for i in range(predictions.shape[0]):
    if predictions[i]>0.5:
        val_preds[i] = 1
    else:
        val_preds[i] = 0
val_trues = test_data_generator.classes

labels = test_data_generator.class_indices.keys()

report = classification_report(val_trues, val_preds, target_names=labels)
print(report) 
print(val_preds)

Found 8 images belonging to 2 classes.
              precision    recall  f1-score   support

      norust       1.00      0.50      0.67         4
        rust       0.67      1.00      0.80         4

    accuracy                           0.75         8
   macro avg       0.83      0.75      0.73         8
weighted avg       0.83      0.75      0.73         8

[[0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
